# ComCam star PSF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import astropy.io.fits as pf
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst.summit.utils.plotting import plot
from lsst.geom import Point2D, Extent2I

from scipy.special import erf
from scipy.optimize import minimize

%matplotlib inline

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTComCam")

In [ ]:
expId = 2024110600252
calexp = butler.get('calexp', detector=4, visit=expId, instrument='LSSTComCam')
x = plot(calexp)

In [ ]:
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = False   
charConfig.detection.minPixels = 100
charConfig.doNormalizedCalibration=False
charTask = CharacterizeImageTask(config=charConfig)

charResult = charTask.run(calexp)
sourceCatalog = charResult.sourceCat
sources = sourceCatalog.asAstropy()
sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
sources.sort('base_CircularApertureFlux_3_0_instFlux', reverse=True)


In [ ]:
half = 10
for source in sources:
    if source['base_CircularApertureFlux_3_0_instFlux'] < 500000:
        x = source['base_SdssCentroid_x']
        y = source['base_SdssCentroid_y']
        break
center = Point2D(x, y)
extent = Extent2I(2*half, 2*half)
cutout = calexp.getCutout(center, extent)
fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.subplots_adjust(wspace=0.5)
im = axs[0].imshow(cutout.image.array, origin='lower')
div = make_axes_locatable(axs[0])
cax = div.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax)
axs[0].set_title(f"ComCam {expId}, Det 4")

axs[1].set_title(f"ComCam {expId}, Y={half}")
axs[1].plot(cutout.image.array[half-1, 0:2*half], marker='x', label='Data')
[Imax, sigmax, sigmay, xoff, yoff] = result.x
ys = []
for ii in range(spot.nx):
    xl = spot.x[ii] - xoff - 0.5
    xh = xl + 1.0
    yl = spot.y[half-1] - yoff - 0.5
    yh = yl + 1.0
    ys.append(Area(xl, xh, yl, yh, sigmax, sigmay, Imax))
FWHMx = sigmax * 2.355 * 0.20
axs[1].plot(spot.x, ys, marker='+', ls='--', color='green', label='2D Gauss')
peak = np.max(cutout.image.array[half, 0:2*half])
print(peak)
axs[1].axhline(peak / 2.0, ls='--', color='black')
axs[1].axvline(xoff - FWHMx * 5.0 / 2.0, ls='--', color='black')
axs[1].axvline(xoff + FWHMx * 5.0 / 2.0, ls='--', color='black')
axs[1].text(1, 25000, f"FWHM = \n{FWHMx:.2f} arcsec")
axs[1].legend()
#plt.ylim(0, 25000)
#plt.ylabel("Flux (electrons)")
#plt.xlabel("Pixels")
#plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_FWHM_Slice_{expId}.png")
#plt.savefig(f"/home/cslage/DATA/ComCam_Star_Core_{expId}.png")

In [ ]:
spot = Array2d(0, 20, 20, 0, 20, 20)
spot.data = cutout.image.array
args = spot
x0 = [7E5, 0.9/2.355, 0.9/2.355, 10.0, 10.0]
result = minimize(FOM, x0, args=args, method='Powell')
result

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=np.linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=np.linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=np.zeros([nx,ny])

def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = np.sqrt(2) * sigmax
    ssigy = np.sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

def FOM(params, args):
    fom = 0.0
    spot = args
    [Imax, sigmax, sigmay, xoff, yoff] = params
    area=np.zeros([spot.nx,spot.ny])
    for ii in range(spot.nx):
        for jj in range(spot.ny):
            xl = spot.x[ii] - xoff - 0.5
            xh = xl + 1.0
            yl = spot.y[jj] - yoff - 0.5
            yh = yl + 1.0
            area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
            fom += np.square(area[ii,jj]-spot.data[ii,jj])
    return fom


In [ ]:
spot.y